In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
from tqdm.notebook import tqdm

import os
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import ncolor, edt
import numpy as np
from pathlib import Path
from numba import njit, float32, int32, vectorize
np.seterr(invalid="ignore")
%matplotlib qt

import time
import random
import matplotlib as mpl

root_dir = '/mnt/sda2/Praneeth/ASTFISH/'

# Data on which Omnipose networks are trained are in EXP-22-BP0394 data directory
DATASET_PATH = root_dir + 'data/EXP-22-BP0394 Analysis code data and models/data/omnipose_data/'

# write the path of the omnipose model file here, it should be 
MODEL_CHECKPOINT_PATH = '../../saved_models/omnipose_mm_old_new_320_long.ckpt'
SAVE_DIR = '../../figures/supplementary'

#MODEL_CHECKPOINT_PATH = CHECKPOINT_PATH + "/MM_OMNI/lightning_logs/version_5/checkpoints/epoch=111-step=559.ckpt"

device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:1


In [3]:
from pathlib import Path
from narsil2.segmentation.datasets import MMData, MMDataTest
from narsil2.segmentation.utils_omni import reconstruct_masks_cpu_omni, format_labels, clean_boundary
from narsil2.segmentation.transformations import OmniTestTransformations, unpadNumpyArray, OmniTransformations
from narsil2.segmentation.networks import Unet
from narsil2.segmentation.losses import OmniLoss
from narsil2.segmentation.lightning_modules import UTrainingModule
from scipy.ndimage.morphology import binary_dilation, binary_erosion
from narsil2.segmentation.transformations import normalize99
from skimage import io
from narsil2.segmentation.utils_omni import flows_to_colors, single_flow_to_color

In [4]:
from matplotlib_scalebar.scalebar import ScaleBar

#### SI fig 1a

In [5]:

train_data = MMData(train=True, transforms=None,
                    dataset_path=DATASET_PATH, flows=True, saved_flows=True,
                    phase_fileformat='*.tif',
                   labels_fileformat="*.npy", mothermachine_data=True)
                   
print("Length of train data", len(train_data))

Length of train data 232


In [6]:
labels_list = ['phase','labels', 'dists', 'flow-y', 'flow-x', 'heat', 'boundary', 
               'binary mask', 'weight']
sample = train_data[128]
print("Filename: ", sample['filename'])
phase = sample['phase']
labels = sample['labels'][0]
dists = sample['labels'][1]
flow_y = sample['labels'][2]
flow_x = sample['labels'][3]
heat = sample['labels'][4]
boundary = sample['labels'][5]
binary_mask = sample['labels'][6]
weight = sample['labels'][7]


def custom_cmap(cmap_str):
    # generates a cmap with white for the background
    cmap = plt.get_cmap(cmap_str)
    greys = plt.get_cmap('Greys', 20)
    cmap_list = [cmap(i) for i in range(cmap.N)]
    #random.shuffle(cmap_list)
    cmap_list = [greys(0)] + cmap_list
    custom_cmap = mpl.colors.LinearSegmentedColormap.from_list('custom_cmap', cmap_list, cmap.N + 1)
    return custom_cmap
    


Filename:  /mnt/sda2/Praneeth/ASTFISH/data/EXP-22-BP0394 Analysis code data and models/data/omnipose_data/phase_train/img_000000037_1.tif


In [7]:

save_filename = SAVE_DIR + '/fig1/phase1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(phase[150:600, 0:600], cmap='gray')
scale_bar = ScaleBar(34.7, "nm", location='lower right')
ax.add_artist(scale_bar)
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)

save_filename = SAVE_DIR + '/fig1/labels1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(labels[150:600, 0:600], cmap=custom_cmap('viridis'), interpolation='nearest')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)

save_filename = SAVE_DIR + '/fig1/dists1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(dists[150:600, 0:600], cmap='turbo', interpolation='nearest')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)


save_filename = SAVE_DIR + '/fig1/flowy1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
flowY_img = single_flow_to_color(flow_y, binary_mask, type='Y')
ax.imshow(flowY_img[150:600, 0:600])
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)

save_filename = SAVE_DIR + '/fig1/flowx1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
flowX_img = single_flow_to_color(flow_x, binary_mask, type='X')
ax.imshow(flowX_img[150:600, 0:600])
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)

save_filename = SAVE_DIR + '/fig1/heat1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(heat[150:600, 0:600], cmap='turbo')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)


save_filename = SAVE_DIR + '/fig1/boundary1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(boundary[150:600, 0:600], cmap=custom_cmap('Spectral'), interpolation='none')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)


save_filename = SAVE_DIR + '/fig1/binary_mask1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(binary_mask[150:600, 0:600],interpolation='none')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)



save_filename = SAVE_DIR + '/fig1/weight1a.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(weight[150:600, 0:600], interpolation='none')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)



#### SI Fig 1b

This is to compare the improvement in speed of the parallel DBSCAN you compiled to work with omnipose reconstructions

In [8]:
model = UTrainingModule.load_from_checkpoint(MODEL_CHECKPOINT_PATH)
model.to(device)
model.eval()

UTrainingModule(
  (model): Unet(
    (down_layers): Sequential(
      (0): ConvBlock(
        (block): Sequential(
          (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): ReLU(inplace=True)
        )
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ConvBlock(
        (block): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): ReLU(inplace=True)
        )
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilat

In [9]:
dataset_path = root_dir + 'data/EXP-22-BV6189 AST FISH 220202/The run/Pos117/phase'

In [11]:
segmenation_parameters = {
    'fileformat': "*.tiff",
    'seg_method': "Omnipose",
    'transforms': OmniTestTransformations(return_tensors=True, shrink_half=False),
    'clean_mask': False,
    'cell_prop_threshold': 0.3,
    'min_size': 400,
    'save_seg_mask': True,
    'save_seg_dir': None,
}
phase_dataset = MMDataTest(dataset_path=dataset_path,
                          phase_fileformat=segmenation_parameters['fileformat'],
                          mothermachine_data=True,
                          transforms=segmenation_parameters['transforms'])

time_slow = []
time_fast = []
for index in range(len(phase_dataset)):
    with torch.no_grad():
        out1 = model(phase_dataset[index]['phase'].unsqueeze(0)[:,:, :, :2048].to(device))
        out2 = model(phase_dataset[index]['phase'].unsqueeze(0)[:,:, :, 2048:].to(device))
        out = torch.cat((out1, out2), dim=3)

    start = time.time()
    generated_mask = reconstruct_masks_cpu_omni(out.cpu().numpy()[0], 
                                    cell_prob_threshold=segmenation_parameters['cell_prop_threshold'],
                                    min_size=segmenation_parameters['min_size'],
                                    device="cuda:1", 
                                    clean_mask=segmenation_parameters['clean_mask'],
                                    fast=False)
    duration = time.time() - start
    time_slow.append(duration)
    print("Duration: ", duration , "s")
print("Slow duration tesed------")
    
for index in range(len(phase_dataset)):
    with torch.no_grad():
        out1 = model(phase_dataset[index]['phase'].unsqueeze(0)[:,:, :, :2048].to(device))
        out2 = model(phase_dataset[index]['phase'].unsqueeze(0)[:,:, :, 2048:].to(device))
        out = torch.cat((out1, out2), dim=3)

    start = time.time()
    generated_mask = reconstruct_masks_cpu_omni(out.cpu().numpy()[0], 
                                    cell_prob_threshold=segmenation_parameters['cell_prop_threshold'],
                                    min_size=segmenation_parameters['min_size'],
                                    device="cuda:1", 
                                    clean_mask=segmenation_parameters['clean_mask'],
                                    fast=True)
    duration = time.time() - start
    time_fast.append(duration)
    print("Duration: ", duration , "s")
print("Fast duration tesed------")

Duration:  1.3638932704925537 s
Duration:  1.2649359703063965 s
Duration:  1.3416898250579834 s
Duration:  1.3653578758239746 s
Duration:  1.332545518875122 s
Duration:  1.3671870231628418 s
Duration:  1.3859891891479492 s
Duration:  1.3833808898925781 s
Duration:  1.5114197731018066 s
Duration:  1.498429536819458 s
Duration:  1.513481855392456 s
Duration:  1.5057852268218994 s
Duration:  1.350205898284912 s
Duration:  1.587470531463623 s
Duration:  1.6028659343719482 s
Duration:  1.4311261177062988 s
Duration:  1.8854620456695557 s
Duration:  1.7539384365081787 s
Duration:  1.8703835010528564 s
Duration:  1.785285234451294 s
Duration:  1.8609983921051025 s
Duration:  1.75179123878479 s
Duration:  1.6413495540618896 s
Duration:  1.9069266319274902 s
Duration:  2.091109275817871 s
Duration:  2.0133395195007324 s
Duration:  2.0453708171844482 s
Duration:  2.195756673812866 s
Duration:  2.2335147857666016 s
Duration:  1.9867403507232666 s
Slow duration tesed------
Duration:  0.76121068000

###### Plotting the times

In [58]:
import seaborn as sns
import pandas as pd

In [59]:
time_data = {'sklearn-DBSCAN': time_slow, 'Parallel-DBSCAN': time_fast}

In [60]:
df = pd.DataFrame(time_data)

In [64]:
save_filename = SAVE_DIR + '/fig1/comparision_1b.svg'

sns.set(style='white')
fig = sns.barplot(data=df, alpha=0.3, edgecolor='black', estimator=np.mean, capsize=0.1, ci="sd")
sns.swarmplot(data=df)
plt.ylabel("Time for reconstruction (s)")
plt.title("Speed comparisions between different reconstructions (n = 30)")
plt.tight_layout()

plt.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.show()



In [65]:
speed_improvement = np.mean(time_slow) / np.mean(time_fast)

In [66]:
print(speed_improvement)

1.9815306411823386


#### Fig 1c Performance comparisions for Omnipose


##### Run these notebook first and extract the metrics from them


1. ../seg_notebooks/Metrics-Unet-MM Data.ipynb
2. ../seg_notebooks/Metrics- Omnipose-MM Data.ipynb. It is still a U-net arch that predicts 4 things instead of one thing

In [67]:
threshold = np.linspace(0.5, 1, 100)

In [68]:
metrics_only_mixed_omnipose = np.array([7.97724068e-01, 7.92854846e-01, 7.88897097e-01, 7.86378622e-01,
       7.84835339e-01, 7.83622026e-01, 7.81888425e-01, 7.80597806e-01,
       7.78956532e-01, 7.78510392e-01, 7.76897311e-01, 7.74958849e-01,
       7.74550855e-01, 7.74224460e-01, 7.72433519e-01, 7.71075726e-01,
       7.70289302e-01, 7.69279361e-01, 7.68176138e-01, 7.67558277e-01,
       7.67127872e-01, 7.66250372e-01, 7.63657153e-01, 7.63341367e-01,
       7.62273729e-01, 7.60519326e-01, 7.59869456e-01, 7.58775175e-01,
       7.58149743e-01, 7.57224798e-01, 7.55478323e-01, 7.55267143e-01,
       7.54255652e-01, 7.52880275e-01, 7.52153575e-01, 7.50733376e-01,
       7.50240445e-01, 7.49680936e-01, 7.48218358e-01, 7.46006668e-01,
       7.45545387e-01, 7.44204938e-01, 7.43796945e-01, 7.42194414e-01,
       7.41666257e-01, 7.39501178e-01, 7.38545835e-01, 7.36005962e-01,
       7.34529197e-01, 7.33008027e-01, 7.31101930e-01, 7.29328036e-01,
       7.26961613e-01, 7.24249423e-01, 7.22235024e-01, 7.19500661e-01,
       7.18100250e-01, 7.15892851e-01, 7.10560858e-01, 7.02046394e-01,
       6.97607577e-01, 6.90566242e-01, 6.84907138e-01, 6.77030385e-01,
       6.67105138e-01, 6.55756652e-01, 6.42387033e-01, 6.24386966e-01,
       6.05724394e-01, 5.82218766e-01, 5.59218407e-01, 5.32602608e-01,
       5.01531839e-01, 4.70987260e-01, 4.31569219e-01, 3.93058598e-01,
       3.51521850e-01, 3.11727166e-01, 2.70794839e-01, 2.28814960e-01,
       1.88821614e-01, 1.48578301e-01, 1.18594766e-01, 9.16354060e-02,
       6.33170679e-02, 4.26181369e-02, 2.77536493e-02, 1.54546527e-02,
       8.82746931e-03, 5.95051702e-03, 2.79201404e-03, 1.40552339e-03,
       9.51599155e-04, 4.77240479e-04, 2.17770052e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
      dtype=np.float32)


In [69]:
metrics_only_ecoli_omnipose = np.array([7.93466091e-01, 7.91965425e-01, 7.90918052e-01, 7.89658487e-01,
       7.88646579e-01, 7.87487209e-01, 7.86194444e-01, 7.85383165e-01,
       7.84678936e-01, 7.83996284e-01, 7.83130944e-01, 7.82415748e-01,
       7.81813860e-01, 7.81061232e-01, 7.80384183e-01, 7.79841185e-01,
       7.79312491e-01, 7.78549552e-01, 7.78054237e-01, 7.77261853e-01,
       7.76229382e-01, 7.75848508e-01, 7.74885356e-01, 7.73895741e-01,
       7.72696197e-01, 7.71302819e-01, 7.69869447e-01, 7.67979443e-01,
       7.66425014e-01, 7.64361024e-01, 7.61542380e-01, 7.58312345e-01,
       7.55655527e-01, 7.52377629e-01, 7.49752283e-01, 7.47062445e-01,
       7.44503498e-01, 7.41605639e-01, 7.38403141e-01, 7.35573113e-01,
       7.32984364e-01, 7.29301333e-01, 7.25979030e-01, 7.22989023e-01,
       7.19728231e-01, 7.16199756e-01, 7.11610436e-01, 7.07687378e-01,
       7.03087270e-01, 6.97460353e-01, 6.91439509e-01, 6.85447335e-01,
       6.78422153e-01, 6.69465780e-01, 6.58973873e-01, 6.47978187e-01,
       6.34874165e-01, 6.20271206e-01, 6.03580117e-01, 5.85392952e-01,
       5.66124499e-01, 5.43224394e-01, 5.19507825e-01, 4.93986517e-01,
       4.63584900e-01, 4.33308214e-01, 4.02998179e-01, 3.68556440e-01,
       3.33509296e-01, 3.00526351e-01, 2.68799096e-01, 2.35494077e-01,
       2.05910146e-01, 1.76521659e-01, 1.50022775e-01, 1.25140384e-01,
       1.02636464e-01, 8.28518346e-02, 6.66559264e-02, 5.25232106e-02,
       4.08651568e-02, 3.07925735e-02, 2.29074005e-02, 1.65028069e-02,
       1.15945730e-02, 8.32168851e-03, 5.65811433e-03, 3.80091276e-03,
       2.32178438e-03, 1.46187539e-03, 8.29185359e-04, 3.79230565e-04,
       1.63532197e-04, 6.06305657e-05, 4.01806647e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
      dtype=np.float32)

In [70]:
labelsize=12
axcol='k'
background_color='k'
#plt.style.use('dark_background')
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.xticks(np.arange(min(threshold), max(threshold)+1, 0.25))
plt.xlim([0.5, 1])
plt.ylim([0, 1])
ax.plot(threshold, metrics_only_mixed_omnipose, color='r',linewidth=2, label='Omnipose-mixed')
ax.plot(threshold, metrics_only_ecoli_omnipose, color='b',linewidth=2, label='Omnipose-E.coli')

ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.tick_params(axis='both', which='minor', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True)
ax.set_ylabel('Jaccard Index', fontsize=labelsize)
ax.set_xlabel('IoU matching threshold', fontsize=labelsize)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.patch.set_alpha(0.0)
#fig.patch.set_facecolor(background_color)
ax.legend()
plt.show()

save_metrics = SAVE_DIR + '/fig1/metrics1c.svg'
fig.savefig(save_metrics)
plt.close(fig)

#### Fig 1d Example input to the U-net

In [71]:
from narsil2.segmentation.datasets import mmDataMultiSpecies

In [72]:
dataset_dir = root_dir + 'data/EXP-22-BP0394 Analysis code data and models/data/unet_data/'
species = ['ecoli', 'mixed']
dataset = mmDataMultiSpecies(dataset_dir, species, includeWeights=True)

In [73]:
sample = dataset[10]
print(sample['phaseFilename'])
phase = sample['phase']
mask = sample['mask']
weights = sample['weights']

/mnt/sda2/Praneeth/ASTFISH/data/EXP-22-BP0394 Analysis code data and models/data/unet_data/ecoli/phase_train/img_000000141_1.tif


In [76]:

save_filename = SAVE_DIR + '/fig1/phase1d.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(phase[100:400, 100:500], cmap='gray')
scale_bar = ScaleBar(65, "nm", location='lower right')
ax.add_artist(scale_bar)
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)



save_filename = SAVE_DIR + '/fig1/binary_mask1d.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(mask[100:400, 100:500],interpolation='none')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)


save_filename = SAVE_DIR + '/fig1/weights1d.svg'
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(weights[100:400, 100:500], cmap='turbo')
ax.set_axis_off()
fig.savefig(save_filename, bbox_inches='tight', pad_inches=0)
plt.close(fig)

#### Fig 1e Performance comparisions for U-net

In [77]:
metrics_only_mixed_unet = np.array([5.2288461e-01, 5.2068090e-01, 5.1858687e-01, 5.1672941e-01,
       5.1540202e-01, 5.1346755e-01, 5.1194388e-01, 5.1024640e-01,
       5.0959224e-01, 5.0868493e-01, 5.0771374e-01, 5.0730634e-01,
       5.0607902e-01, 5.0513774e-01, 5.0302607e-01, 5.0202250e-01,
       5.0112402e-01, 5.0046277e-01, 4.9962589e-01, 4.9855608e-01,
       4.9618116e-01, 4.9570736e-01, 4.9475265e-01, 4.9423116e-01,
       4.9299863e-01, 4.9202961e-01, 4.9090275e-01, 4.8991060e-01,
       4.8921254e-01, 4.8911577e-01, 4.8878056e-01, 4.8812020e-01,
       4.8741728e-01, 4.8647201e-01, 4.8510170e-01, 4.8453176e-01,
       4.8321950e-01, 4.8182929e-01, 4.8088753e-01, 4.7673717e-01,
       4.7457737e-01, 4.7343621e-01, 4.7257912e-01, 4.7193828e-01,
       4.6985981e-01, 4.6876591e-01, 4.6693203e-01, 4.6381107e-01,
       4.6175611e-01, 4.5970675e-01, 4.5802036e-01, 4.5571959e-01,
       4.5235905e-01, 4.4668263e-01, 4.4289562e-01, 4.3890259e-01,
       4.3400201e-01, 4.3109876e-01, 4.2653126e-01, 4.2169762e-01,
       4.1681838e-01, 4.0672925e-01, 3.9795712e-01, 3.8736984e-01,
       3.7705666e-01, 3.6672688e-01, 3.5614827e-01, 3.3762243e-01,
       3.1910616e-01, 3.0079806e-01, 2.8038365e-01, 2.6043004e-01,
       2.3283353e-01, 2.0559709e-01, 1.8206233e-01, 1.5499112e-01,
       1.3112368e-01, 1.0652599e-01, 8.7873794e-02, 7.1041606e-02,
       5.2920137e-02, 3.8193792e-02, 2.7532693e-02, 1.8327355e-02,
       1.1181131e-02, 6.7662159e-03, 3.6521319e-03, 1.6192833e-03,
       6.7354477e-04, 1.7454584e-04, 3.2870947e-05, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
      dtype=np.float32)

metrics_only_ecoli_unet = np.array([8.4449577e-01, 8.4252995e-01, 8.4091908e-01, 8.3966106e-01,
       8.3847070e-01, 8.3721507e-01, 8.3613849e-01, 8.3468711e-01,
       8.3361250e-01, 8.3223617e-01, 8.3140713e-01, 8.3035177e-01,
       8.2918400e-01, 8.2772183e-01, 8.2686520e-01, 8.2597291e-01,
       8.2516187e-01, 8.2380491e-01, 8.2266355e-01, 8.2165515e-01,
       8.2083720e-01, 8.2005692e-01, 8.1931657e-01, 8.1812811e-01,
       8.1736636e-01, 8.1636596e-01, 8.1528038e-01, 8.1439859e-01,
       8.1322151e-01, 8.1207299e-01, 8.1061041e-01, 8.0861288e-01,
       8.0673927e-01, 8.0474049e-01, 8.0271411e-01, 8.0077726e-01,
       7.9810005e-01, 7.9564703e-01, 7.9288024e-01, 7.9058939e-01,
       7.8827804e-01, 7.8592408e-01, 7.8353417e-01, 7.8123701e-01,
       7.7880347e-01, 7.7621555e-01, 7.7365452e-01, 7.7146101e-01,
       7.6962918e-01, 7.6712281e-01, 7.6427621e-01, 7.6110393e-01,
       7.5788790e-01, 7.5534713e-01, 7.5140446e-01, 7.4809700e-01,
       7.4342531e-01, 7.3805374e-01, 7.3327392e-01, 7.2769493e-01,
       7.2022748e-01, 7.1285808e-01, 7.0362872e-01, 6.9269890e-01,
       6.7934167e-01, 6.6392374e-01, 6.4825940e-01, 6.2849987e-01,
       6.0678476e-01, 5.8061963e-01, 5.5462825e-01, 5.2447766e-01,
       4.9312913e-01, 4.5942536e-01, 4.2408893e-01, 3.8554046e-01,
       3.5002753e-01, 3.1504899e-01, 2.7861014e-01, 2.4162132e-01,
       2.0687240e-01, 1.7396659e-01, 1.4296693e-01, 1.1347832e-01,
       8.8517338e-02, 6.7155957e-02, 4.9282670e-02, 3.4632523e-02,
       2.3479717e-02, 1.4742984e-02, 8.8721374e-03, 5.1641879e-03,
       2.6860170e-03, 1.3093709e-03, 5.6244986e-04, 1.4231942e-04,
       6.7977424e-05, 3.3341650e-05, 0.0000000e+00, 0.0000000e+00],
      dtype=np.float32)

In [78]:
labelsize=12
axcol='k'
background_color='k'
#plt.style.use('dark_background')
fig, ax = plt.subplots(1,1, figsize=(5,5))
plt.xticks(np.arange(min(threshold), max(threshold)+1, 0.25))
plt.xlim([0.5, 1])
plt.ylim([0, 1])
ax.plot(threshold, metrics_only_mixed_unet, color='r',linewidth=2, label='Unet-mixed')
ax.plot(threshold, metrics_only_ecoli_unet, color='b',linewidth=2, label='Unet-E.coli')

ax.tick_params(axis='both', which='major', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True,left=True)
ax.tick_params(axis='both', which='minor', labelsize=labelsize,length=3, direction="out",colors=axcol,bottom=True)
ax.set_ylabel('Jaccard Index', fontsize=labelsize)
ax.set_xlabel('IoU matching threshold', fontsize=labelsize)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.patch.set_alpha(0.0)
#fig.patch.set_facecolor(background_color)
ax.legend()
plt.show()

save_metrics = SAVE_DIR + '/fig1/metrics1e.svg'
fig.savefig(save_metrics)
plt.close(fig)


### 